<a href="https://colab.research.google.com/github/dkshjn/Technoculture/blob/main/DSPy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# @title Initial Setup
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    # !pip install -e $repo_path

import dspy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Already up to date.


In [37]:
import openai

openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = "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"

temperature_condition = 0.2

temperature = temperature_condition if temperature_condition is not None else 0.2
response = openai.ChatCompletion.create(
  model="openai/gpt-3.5-turbo", # Optional (user controls the default)
  messages=[
      {"role": "user", "content": "How do i make tea?"}
    ],
      kwargs={
        "temperature": temperature
    },
  # headers={
  #   "HTTP-Referer": $YOUR_SITE_URL, # Optional, for including your app on openrouter.ai rankings.
  #   "X-Title": $YOUR_APP_NAME, # Optional. Shows in rankings on openrouter.ai.
  # },
)

reply = response.choices[0].message
print(reply.content)

To make tea, follow these general steps:

1. Boil water: Fill a kettle with fresh water and put it on the stove to heat until it reaches a rolling boil.
2. Choose your tea: Select the type of tea you want to make - black tea, green tea, white tea, herbal tea, or any other variety.
3. Warm the teapot/mug: Pour hot water into the teapot or the mug you plan to use to warm it up. Let it sit for a moment, then discard the water.
4. Add tea leaves: If using loose tea leaves, add about one teaspoon per cup of boiling water to the teapot or a tea infuser. If using teabags, place one tea bag per cup into the teapot or mug.
5. Pour the water: Pour the boiling water into the teapot or mug over the tea leaves or teabags.
6. Steep the tea: Let the tea leaves or teabags steep in the water for the appropriate time. This time will vary depending on the type of tea you are brewing, usually 3-7 minutes.
7. Remove tea leaves/teabags: Once the tea has steeped enough, remove the tea leaves or teabags from 

In [38]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
temperature = response.choices[0].message['temperature'] if 'temperature' in response.choices[0].message else 0.7
dspy.settings.configure(lm=response, rm=colbertv2_wiki17_abstracts, temperature=temperature)

In [39]:
from dspy.datasets import HotPotQA

dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]
len(trainset), len(devset)

(20, 50)

In [40]:
train_example1 = trainset[0]
print(f"Question1: {train_example1.question}")
print(f"Answer1: {train_example1.answer}\n")

train_example2 = trainset[1]
print(f"Question2: {train_example2.question}")
print(f"Answer2: {train_example2.answer}\n")

train_example3 = trainset[2]
print(f"Question3: {train_example3.question}")
print(f"Answer3: {train_example3.answer}\n")

Question1: At My Window was released by which American singer-songwriter?
Answer1: John Townes Van Zandt

Question2: which  American actor was Candace Kita  guest starred with 
Answer2: Bill Murray

Question3: Which of these publications was most recently published, Who Put the Bomp or Self?
Answer3: Self



In [41]:
dev_example1 = devset[15]
print(f"Question: {dev_example1.question}")
print(f"Answer: {dev_example1.answer}")
print(f"Relevant Wikipedia Titles: {dev_example1.gold_titles}\n")

dev_example2 = devset[16]
print(f"Question: {dev_example2.question}")
print(f"Answer: {dev_example2.answer}")
print(f"Relevant Wikipedia Titles: {dev_example2.gold_titles}\n")

dev_example3 = devset[18]
print(f"Question: {dev_example3.question}")
print(f"Answer: {dev_example3.answer}")
print(f"Relevant Wikipedia Titles: {dev_example3.gold_titles}\n")

Question: What is the name of this region of Italy, referring to the medieval March of Ancona and nearby marches of Camerino and Fermo, where the comune Pollenza is located?
Answer: Marche
Relevant Wikipedia Titles: {'Marche', 'Pollenza'}

Question: William Hughes Miller was born in a city with how many inhabitants ?
Answer: 7,402 at the 2010 census
Relevant Wikipedia Titles: {'Kosciusko, Mississippi', 'William Hughes Miller'}

Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
Answer: English
Relevant Wikipedia Titles: {'Robert Irvine', 'Restaurant: Impossible'}



In [42]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [43]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [44]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [50]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)

# Compile!
compiled_rag = teleprompter.compile(RAG(), trainset=trainset)

 20%|██        | 4/20 [00:00<00:00, 718.02it/s]

Failed to run or to evaluate example Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x7a9f7aaa7250> due to kwargs.
Failed to run or to evaluate example Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x7a9f7aaa7250> due to kwargs.
Failed to run or to evaluate example Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}) with <function validate_context_and_answer at 0x7a9f7aaa7250> due to kwargs.
Failed to run or to evaluate example Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?', 'answer': '1950'}) (input_keys={'question'}) with <func

AttributeError: ignored

In [ ]:
# Ask any question you like to this simple RAG program.
my_question = "What castle did David Gregory inherit?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")